# Teste do scrapper

O primeiro modelo de scrapper sera com selenium, pois é preciso criar um mecanismo para inicial para alimentar o Datalake. Nesse script, a autenticação do captcha será manual e a coleta das tabelas com os dados serão extraídas pela aplicação e carregadas no Atlas.

O código a seguir o `selenium` faz a carga da pagina de busca e para na autenticação do CAPTCHA.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

import time
import json

In [2]:
nire_list = []
driver = webdriver.Firefox(executable_path = r'.\webdriver\geckodriver.exe')

driver.get('https://www.jucesponline.sp.gov.br/BuscaAvancada.aspx?IDProduto=')
txtMunicipio = driver.find_element_by_id('ctl00_cphContent_frmBuscaAvancada_txtMunicipio')
txtMunicipio.send_keys('santos')

btPesquisar = driver.find_element_by_id('ctl00_cphContent_frmBuscaAvancada_btPesquisar')
btPesquisar.click()

O próximo passo é autenticar o CAPTCHA e aguardar a página com os resultados.

Os dados estão em uma tabela identificada com o seguinte tag.

`<table rules="all" id="ctl00_cphContent_gdvResultadoBusca_gdvContent" style="border-width:0px;width:100%;" cellspacing="1" cellpadding="4" border="0">`

Utiliza-se um explicit wait para fazer com que o script aguarde o usuário inserir o CAPTCHA e a tabela de dados carregar.

In [3]:
data_table = WebDriverWait(driver,30).until(EC.presence_of_element_located((By.ID,'ctl00_cphContent_gdvResultadoBusca_gdvContent')))

Verificar como o selenium se comporta com o vai e vem das páginas.

A página tem um timeout e de tempos em tempos ela pede outro captcha.

Para pegar os documentos do detalhamento precisa de logon.

Como o selenium se perde quando ele avança para a página de detalhamento e depois retona para a lista de NIREs, a melhor abordagem é percorrer a lista e coletar todos os números existentes, para depois acessar as páginas individualmente. Haja visto, existe a possibilidade de acesso direto aos dados através da URL - `https://www.jucesponline.sp.gov.br/Pre_Visualiza.aspx?nire=<NIRE>`

In [4]:
raw_data = data_table.find_elements_by_tag_name('td')
for nire in raw_data:
    if nire.get_attribute("class") == 'item01':
        nire_list.append(nire.text.strip())  

## Navegação entre as páginas com resultados.


Como condição de parada o cógigo deve verificar se a quantidade de itens exibidos bate com a quantidade de itens encontrados, para isso, se utiliza a informação existente junto aos botões de navegação.


In [11]:
result_count = int(driver.find_element(By.ID,'ctl00_cphContent_gdvResultadoBusca_pgrGridView_lblResultCount').text.replace('.', ''))
btNext = driver.find_element(By.ID,'ctl00_cphContent_gdvResultadoBusca_pgrGridView_btrNext_lbtText')
result = int(driver.find_element(By.ID,'ctl00_cphContent_gdvResultadoBusca_pgrGridView_lblResults').text.replace('.','')[-1])
if (result < result_count):
    time.sleep(10)
    btNext.click()
    loading = WebDriverWait(driver,5).until(EC.invisibility_of_element((By.ID,'ctl00_cphContent_gdvResultadoBusca_qtpLoading_pnlMessage')))
    print(loading)
else:
    pass

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="a598e525-b8c2-43e4-bbb8-81b6feca7ba8", element="d6ce9d96-cc05-467a-b099-429d518cff11")>


Mesmo depois dos ajustes o script ainda dá excessões de Staled Element. Talvez seja melhor processa a página offline.

O código abaixo grava o HTML no disco para inspeção.

In [4]:
file = open('source.html','w')
file.write(driver.page_source)
file.close()

## Processa a página com os detalhamentos


O código a seguir acessa as páginas com os dados detalhados das empresas e faz o dump deles em um aquivo no formato json.

As páginas são acessadas diretamente através da URL base `https://www.jucesponline.sp.gov.br/Pre_Visualiza.aspx?nire=` mais o código NIRE recuperado no passo anterior.

Abaixo seguem as tags da página de detalhamento da empresa com dados relevantes para a aplicação.

Área com os dados da empresa:  
`<div id="dados">`

Timestamp da query:  
`<p class="antetitulo"> Data de emissão: <span id="ctl00_cphContent_frmPreVisualiza_lblEmissao">26/08/2020 18:27:24</span></p>`

Nome da empresa:  
`<span id="ctl00_cphContent_frmPreVisualiza_lblEmpresa" class="titulo-azul16-01">ESTRELA COMERCIO DE UTILIDADES LTDA</span>`

Tipo de Empresa:  
`<span id="ctl00_cphContent_frmPreVisualiza_lblDetalhes">LIMITADA UNIPESSOAL (E.P.P.)</span>`

Data da Constituição:  
`<span id="ctl00_cphContent_frmPreVisualiza_lblConstituicao">26/08/2020</span>`

Início da Atividade:  
`<span id="ctl00_cphContent_frmPreVisualiza_lblAtividade">20/08/2020</span>`

CNPJ:  
`<span id="ctl00_cphContent_frmPreVisualiza_lblCnpj">38.231.226/0001-76</span>`

Inscrição Estadual:  
`<span id="ctl00_cphContent_frmPreVisualiza_lblInscricao"></span>`

Objetivo:  
`<span id="ctl00_cphContent_frmPreVisualiza_lblObjeto">Comércio varejista de outros artigos de uso doméstico não especificados anteriormente<br>Comércio varejista de cosméticos, produtos de perfumaria e de higiene pessoal<br>Comércio varejista de artigos de papelaria<br>Comércio varejista de brinquedos e artigos recreativos<br>Comércio atacadista de outros equipamentos e artigos de uso pessoal e doméstico não especificados anteriormente</span>`

Capital:  
`<span id="ctl00_cphContent_frmPreVisualiza_lblCapital">R$       30.000,00 (Trinta Mil Reais)</span>`

Logradouro:  
`<span id="ctl00_cphContent_frmPreVisualiza_lblLogradouro">Rua Julio Conceicao</span>`

Numero:  
`<span id="ctl00_cphContent_frmPreVisualiza_lblNumero">257</span>`

Bairro:  
`<span id="ctl00_cphContent_frmPreVisualiza_lblBairro">Vila Mathias</span>`

Complemento:  
`<span id="ctl00_cphContent_frmPreVisualiza_lblComplemento"></span>`

Município:  
`<span id="ctl00_cphContent_frmPreVisualiza_lblMunicipio">Santos</span>`

CEP:  
`<span id="ctl00_cphContent_frmPreVisualiza_lblCep">11015-540</span>`

Unidade Federativa:  
`<span id="ctl00_cphContent_frmPreVisualiza_lblUf">SP</span>`



In [11]:
url_base = 'https://www.jucesponline.sp.gov.br/Pre_Visualiza.aspx?nire='
empresas = []
for nire in nire_list:
    driver.get(url_base + nire)
    
    timestamp         = driver.find_element(By.ID,"ctl00_cphContent_frmPreVisualiza_lblEmissao").text
    nome              = driver.find_element(By.ID, "ctl00_cphContent_frmPreVisualiza_lblEmpresa").text
    tipo              = driver.find_element(By.ID, "ctl00_cphContent_frmPreVisualiza_lblDetalhes").text
    data_constituicao = driver.find_element(By.ID, "ctl00_cphContent_frmPreVisualiza_lblConstituicao").text
    inicio_atividade  = driver.find_element(By.ID, "ctl00_cphContent_frmPreVisualiza_lblAtividade").text
    cnpj              = driver.find_element(By.ID, "ctl00_cphContent_frmPreVisualiza_lblCnpj").text
    ie                = driver.find_element(By.ID, "ctl00_cphContent_frmPreVisualiza_lblInscricao").text
    objeto            = driver.find_element(By.ID, "ctl00_cphContent_frmPreVisualiza_lblObjeto").text
    capital           = driver.find_element(By.ID, "ctl00_cphContent_frmPreVisualiza_lblCapital").text
    logradouro        = driver.find_element(By.ID, "ctl00_cphContent_frmPreVisualiza_lblLogradouro").text
    numero            = driver.find_element(By.ID, "ctl00_cphContent_frmPreVisualiza_lblNumero").text
    bairro            = driver.find_element(By.ID, "ctl00_cphContent_frmPreVisualiza_lblBairro").text
    complemento       = driver.find_element(By.ID, "ctl00_cphContent_frmPreVisualiza_lblComplemento").text
    municipio         = driver.find_element(By.ID, "ctl00_cphContent_frmPreVisualiza_lblMunicipio").text
    cep               = driver.find_element(By.ID, "ctl00_cphContent_frmPreVisualiza_lblCep").text
    uf                = driver.find_element(By.ID, "ctl00_cphContent_frmPreVisualiza_lblUf").text

    empresas.append({'timestamp':timestamp, 'nire':nire, 'nome':nome,'tipo':tipo,'data_constituicao':data_constituicao, 'inicio_atividade':inicio_atividade, 'cnpj':cnpj, 'ie':ie, 'objeto':objeto, 'capital':capital,'logradouro':logradouro, 'numero':numero, 'bairro':bairro, 'complemento':complemento, 'municipio':municipio, 'cep':cep, 'uf':uf})
    time.sleep(5)

f = open('data_file.json','w')
f.write(json.dumps(empresas))

10103

In [ ]:
O algoritmo anterior sofre com exceções aleatórias devido ao lag na carga da página. Vou tentar baixar a página toda para o disco e processar offline.



# Teste do OCR

O módulo `pillow` é uma biblioteca para a manipulação de imagens, ela pemite a carga e ajustem em imagens de diversos formatos.

O módulo pytesseract é um `wrapper` para o software de reconhecimento de imagens `tesseract` que tem que ser instalado a parte.


O código a seguir carrega as bibliotecas e configura o caminho para o executável do tesseract.


In [ ]:
from PIL import Image
from PIL import ImageFilter

import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'D:\Tools\Tesseract-OCR\tesseract'

O trecho a seguir carrega na memória a imagem abaixo e a exibe em na aplicação de visualização de imagens do sistema operacional.

![Imagem de Teste](images\test.jpg)


In [ ]:
captcha = Image.open(r'images\test.jpg')
captcha.show()

A próxima linha analisa a imagem e imprime o texto extraído na tela. Caso nada tenha sido encontrado, não há saída para a tela.


In [ ]:
print(pytesseract.image_to_string(captcha))

Com uma imagem limpa o reconhecimento funciona sem grandes problemas. Vamos testar agora com a seguinte imagem coletada do CAPTCHA da Jussesp.

![CAPTCHA Jussesp](images\CaptchaImage.aspx.jfif)


In [ ]:
captcha = Image.open(r'images\CaptchaImage.aspx.jfif')
print(pytesseract.image_to_string(captcha))

Devido ao ruído de fundo o OCR não foi capaz de reconhecer nenhum dos caracteres da imagem.

Vou tentar tratar a imagem para diminuir o nível de ruído.


In [ ]:
#filterImg = captcha.filter(ImageFilter.SHARPEN)
filterImg = captcha.filter(ImageFilter.FIND_EDGES)
filterImg.show()
print(pytesseract.image_to_string(filterImg))

Remover as "pintas" do fundo da imagem e passar pelo OCR novamente.

![CAPTCHA Tratado 1](images\CaptchaImage13T.aspx.jpg)


In [ ]:
captcha = Image.open(r'images\CaptchaImage13T.aspx.jpg')
print(pytesseract.image_to_string(captcha))